# TODO:
#### Automatic data retrieval from google forms for during-suvrvey checking
[X] spreadsheets url inference <br>
[X] gather results in python script

#### Qualification filtering
[X] ~50 qualifications

#### Frauder detection
[X] > X ratio repeated ===> discarded

#### Honeypots
[X] place honeypots from a defined set in each form<br>
[X] honeypot frauders detection

#### MTurk templating
[X] Create guidelines for the workers

#### Gform templating
[X] Remove index of the emojis and add separator btwn each question NB: not practical

#### Emojis png
[X] Rerun creation of emojis png + upload on drive

___

#### MTurk features
[X] Implement dry runs <br>
[ ] Link between MTurk framework and gdrive service (not same ids etc) ==> visualization of the actual results for a given workerid + hitid (not just the completion code as before)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import create_mturk_client,clean_own_worker,Turker,MTurkparam
from src.data.watcher import Watcher
from src.visualization.turk_widgets import get_control_panel
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results,download_drive_spreadsheet
from src.data.fraudulous import detect_repeat_frauders,detect_honey_frauders
from src.utils import generate_password 
from src.constants import WATCHER_PATH,NMB_FORMS_THRESHOLD,TOKEN_PATH,CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH, FORMS_RESULTS_DIR,WATCHER_FORMS_RESULTS_DIR,HONEYPOTS
from src.utils import read_access_keys
import subprocess
import pandas as pd
import jdc
from IPython.display import clear_output
import pickle as pk
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

import ipywidgets as widgets
from src.utils import get_form_urls


## Urls info retrieval

In [4]:
# does not work: the id is not the right one
#response = service.files().list(q="name = 'forms_url' and trashed=false").execute()
# files = response['files']
#file_id = files[0]['id']

In [10]:
file_id = "16oN7FCdGsvnRCR1R0DYQLPFyZarSOlDbPfSq2-Bxc80"
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
formidx2url,formidx2gid = get_form_urls(service,file_id)
download_all_csv_results(formidx2gid,FORMS_RESULTS_DIR,service)

Download 100%


## MTurk management

[ ] Monitor > 2 <br>
[ ] Honeypots <br>
[ ] Repeat <br>
[ ] Fake id <br>
[ ] Dry runs

In [11]:
param = MTurkparam(hitlayout="3VQCRCGMCT2NU7RBSKK6J2PIFKVQ77",
             MaxAssignments = 10,
             LifetimeInSeconds = 600,
             AutoApprovalDelayInSeconds=600,
             AssignmentDurationInSeconds=600,
             Reward='0.01',
             n_forms=len(formidx2gid),
             max_forms_per_worker=1)

Estimated cost:0.30 $


In [17]:
turk = Turker(param=param,
              gservice=service,
              formidx2url=formidx2url,
              formidx2gid=formidx2gid,
              formrespath=FORMS_RESULTS_DIR)

Loading hit2form


In [13]:
get_control_panel(turk,file_id)

Text(value='', placeholder='Results HITid/formidx')

In [ ]:
turk.approve_correct_assignments("32TMVRKDGNJ1OLXQZYKUBCMAKDL48T",dry_run=True)

> /home/ymentha/Documents/Cours/dlab_project/emojivec/src/data/auto_mturk.py(299)approve_correct_assignments()
    297         fakeid_frauders = workers - set(form_df['Worker ID'].unique().tolist()) # workers who entered a fake id
    298         set_trace()
--> 299         frauders = (password_frauders.union(honey_frauders)
    300                                     .union(repeat_frauders)
    301                                     .union(fakeid_frauders))



ipdb>  honey_frauders


{'A29C1XYH77RQYM'}


ipdb>  l


    294         # Manually entered MTurk ids
    295         honey_frauders = detect_honey_frauders(form_df,HONEYPOTS) # wrong honeypot
    296         repeat_frauders = detect_repeat_frauders(form_df) # repeat words
    297         fakeid_frauders = workers - set(form_df['Worker ID'].unique().tolist()) # workers who entered a fake id
    298         set_trace()
--> 299         frauders = (password_frauders.union(honey_frauders)
    300                                     .union(repeat_frauders)
    301                                     .union(fakeid_frauders))
    302 
    303         for assignment in assignments:
    304             ass_id = assignment['AssignmentId']



...

0    sun
Name: ☀️, dtype: object


ipdb>  l


    305             worker_id = assignment['WorkerId']
    306             if worker_id in frauders:
    307                 RequesterFeedback = ""
    308                 if worker_id in password_frauders:
    309                     RequesterFeedback += "Invalid confirmation key.\n"
    310 
    311                 if worker_id in honey_frauders:
    312                     RequesterFeedback += "* Non valid obvious emoji answer.\n"
    313 
    314                 if worker_id in repeat_frauders:
    315                     RequesterFeedback += "* Too many times the same word.\n"



ipdb>  honey_frauders


{'A29C1XYH77RQYM'}


https://workersandbox.mturk.com/mturk/

In [15]:
clean_own_worker(turk.client)

Worker already clean


## Qualifications

In [ ]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=0
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

In [ ]:
turker.client.update_qualification_type(
    QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN',
    QualificationTypeStatus='Active',
    Test='string',
    AnswerKey='string',
    TestDurationInSeconds=123,
    RetryDelayInSeconds=123,
    AutoGranted=False,
    AutoGrantedValue=123
)